In [7]:
%pip install "datasets==1.6.2"
%pip install transformers
%pip install --upgrade sagemaker
%pip install ipywidgets

distutils: /opt/conda/include/python3.6m/UNKNOWN
sysconfig: /opt/conda/include/python3.6m
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.6m/UNKNOWN
sysconfig: /opt/conda/include/python3.6m
user = False
home = None
root = None
prefix = None
Note: you may need to restart the kernel to use updated packages.
distutils: /opt/conda/include/python3.6m/UNKNOWN
sysconfig: /opt/conda/include/python3.6m
user = False
home = None
root = None
prefix = None
  Using cached transformers-4.9.2-py3-none-any.whl (2.6 MB)
  Using cached sacremoses-0.0.45-py3-none-any.whl (895 kB)
  Using cached huggingface_hub-0.0.12-py3-none-any.whl (37 kB)
  Using cached tokenizers-0.10.3-cp36-cp36m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
  Using cached regex-2021.8.21-cp36-cp36m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (748 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0

In [1]:
local = False
debug = False

In [2]:
from datasets import Dataset
import datasets
import pandas as pd
from sklearn.model_selection import train_test_split
import utils


In [3]:
import roberta_model
from types import SimpleNamespace
from transformers import file_utils

In [4]:
if not local:
    import sagemaker
    from datasets.filesystems import S3FileSystem
    sess = sagemaker.Session()
    role = sagemaker.get_execution_role()
    s3_bucket = sess.default_bucket()

In [5]:
cfg = SimpleNamespace()
cfg.kw_neurons = 10
cfg.kw_dropout = 0.1
cfg.clf_neurons = 5
cfg.clf_dropout = 0.1
cfg.freeze_roberta_params = True
cfg.hf_roberta_model = 'distilroberta-base'

model = roberta_model.DisasterModel(vars(cfg))

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
df = pd.read_csv('../train.csv').loc[:,['keyword','text','target']]

test_size = 0.1

if debug:
    df = df.head(128)
    test_size=0.5

df = utils.preprocess_df(df)

train, test = train_test_split(df, test_size=test_size, random_state=42)

features=datasets.Features({
    "keywords": datasets.Value("string"),
    "text": datasets.Value("string"),
    "label": datasets.ClassLabel(num_classes=2, names=['non-disaster', 'disaster'])}
)

In [7]:
tokenizer = model.tokenizer

train_ds = Dataset.from_pandas(train, features=features).map(
    utils.tokenize_fn,
    fn_kwargs={'tokenizer': tokenizer},
    batched=True
)
test_ds = Dataset.from_pandas(test, features=features).map(
    utils.tokenize_fn,
    fn_kwargs={'tokenizer': tokenizer},
    batched=True
)

In [9]:
if not local:
    s3 = S3FileSystem()
    train_ds.save_to_disk(f's3://{s3_bucket}/tweet_and_tag/train', fs=s3)
    test_ds.save_to_disk(f's3://{s3_bucket}/tweet_and_tag/test', fs=s3)
    sess.list_s3_files(s3_bucket, key_prefix='dataset')
else:
    train_ds.save_to_disk('dataset/train')
    test_ds.save_to_disk('dataset/test')